In [1]:
%%capture
# %%capture: Show now output for this cell
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

# https://pydata.github.io/pandas-datareader/
!pip install pandas-datareader
import pandas_datareader as pdr

# Implement stock predictor
For your core stock predictor, implement:

- A training interface that accepts a data range (start_date, end_date) and a list of ticker symbols (e.g. GOOG, AAPL), and builds a model of stock behavior. Your code should read the desired historical prices from the data source of your choice.
- A query interface that accepts a list of dates and a list of ticker symbols, and outputs the predicted stock prices for each of those stocks on the given dates. Note that the query dates passed in must be after the training date range, and ticker symbols must be a subset of the ones trained on.

In [36]:
# Define a date range
start = dt.datetime(2018, 1, 29)
end = dt.datetime.today()

# Define ticker symbols
ticker_symbols = ['GOOG', "AAPL"]

In [ ]:
def get_stock_data(ticker_symbols, start_date, end_date = dt.datetime.today(), date_index = False, columns="all"):
    '''
    Returns stock data from yahoo for the specified ticker symbols, start and end dates.
    A dictionary is returned and the data for each stock can be accessed by data["ticker_symbol"]
    '''
    ticker_symbols = set(ticker_symbols)
    data = {}
    for ticker in ticker_symbols:
        df = pdr.data.DataReader(ticker, 'yahoo', start = start_date, end = end_date)
        if date_index:
            if "Date" in df.columns:
                df.set_index = df["Date"]
        else:
            df["Date"] = df.index
            df.index =  range(1, df.shape[0] + 1)
        if columns != "all":
            if len(columns) > 0:
                for column in df.columns:
                    if column not in columns:
                        df = df.drop([column], axis = 1)
        if df.isnull().sum().sum() != 0:
            print(f"WARN: {df.isnull().sum().sum() } data points are missing for ticker symbol {ticker}")
        data[ticker] = df
    return data

In [19]:
#get_stock_data(ticker_symbols, start)

In [10]:
def get_model(name):
    model_dict = {}
    model_dict["name"] = name
    
    return model_dict

In [11]:
def train_model(model_dict, ticker_symbols, start_date, end_date = dt.datetime.today()):
    ticker_symbols = set(ticker_symbols)
    data = get_stock_data(ticker_symbols, start_date, end_date)
    
    model_dict["ticker_symbols"] = ticker_symbols
    model_dict["start_date"] = start_date
    model_dict["end_date"] = end_date
    
    return model_dict   

In [34]:
def make_prediction(model_dict, dates, ticker_symbols):
    '''
    
    '''
    
    # Check if the model was trained for all the provided ticker symbols:
    ticker_symbols = set(ticker_symbols)
    for ticker in ticker_symbols:
        if ticker not in model_dict["ticker_symbols"]:
            print(f"WARN: model {model['name']} was not trained to predict prices for {ticker}!")
            ticker_symbols.drop(ticker)
            
    # Check if the date ranges are outside the range which were used for training:
    for date in dates:
        if (date > model["start_date"]) and (date < mode["end_date"]):
            print(f"WARN: the model was trained with the date {date} and therefore no predictions for that date can be made!")
            dates.drop(date)
            
    # Initialize a result dictionary
    result = {}
    for ticker in ticker_symbols:
        for date in dates:
            prediction = 0
            result[ticker][date] = prediction
        
    return result

# Test and measure performance
A basic run of the core system would involve one call to the training interface, and one or more calls to the query interface. Implement a train-test cycle to measure the performance of your model. Use it to test prediction accuracy for query dates at different intervals after the training end date, e.g. the day immediately after training end date, 7 days later, 14 days, 28 days, etc.

(Note: Pick the training period accordingly so that you have ground truth data for that many days in the future.)